
# Google Form → responses_processed.csv & .jsonl

This notebook converts **Google Form** responses (CSV from Google Sheets) into a **processed dataset** that matches the pipeline schema, including:

- Normalizing **Likert 1–5 → 0–1**.
- Averaging into **RIASEC (R,I,A,S,E,C)** and **Big Five (O,C,E,A,N)**.
- Extracting **essay_text** and **target_jobs**.
- Adding **instruction fields** (`riasec_instructions`, `big5_instructions`).
- Exporting to both **CSV** and **JSONL** (one JSON record per line).

> ✅ Edit paths and column titles in **Config** cell to match your Form exactly.


In [ ]:
# === Config ===

# Path to CSV exported from Google Sheets (File -> Download -> CSV)
INPUT_CSV = "form_responses.csv"  # <-- change to your file path
OUTPUT_CSV = "responses_processed.csv"
OUTPUT_JSONL = "responses_processed.jsonl"

# Instruction fields (will be included in output columns, same value for every row)
RIASEC_INSTRUCTIONS = (
    "Câu hỏi RIASEC (12 câu) - Trả lời theo thang Likert: "
    "1 = Rất không thích, 2 = Không thích, 3 = Trung lập, 4 = Thích, 5 = Rất thích"
)
BIG5_INSTRUCTIONS = (
    "Câu hỏi Big Five (10 câu) - Trả lời theo thang Likert: "
    "1 = Rất không đồng ý, 2 = Không đồng ý, 3 = Trung lập, 4 = Đồng ý, 5 = Rất đồng ý"
)

# === Column titles in your Google Form ===

# RIASEC 12 (2 per trait)
R_cols = [
    "Tôi thích sửa chữa hoặc lắp ráp các thiết bị. (R)",
    "Tôi muốn làm việc ngoài trời, vận động chân tay. (R)",
]
I_cols = [
    "Tôi thích nghiên cứu, tìm hiểu các vấn đề phức tạp. (I)",
    "Tôi hứng thú với việc giải quyết bài toán khoa học/kỹ thuật. (I)",
]
A_cols = [
    "Tôi thích vẽ, viết, hoặc thể hiện ý tưởng sáng tạo. (A)",
    "Tôi hứng thú với nghệ thuật, âm nhạc, hoặc sáng tác. (A)",
]
S_cols = [
    "Tôi thích giúp đỡ, lắng nghe và hỗ trợ người khác. (S)",
    "Tôi muốn tham gia các hoạt động cộng đồng, nhóm. (S)",
]
E_cols = [
    "Tôi thích lãnh đạo, điều phối công việc nhóm. (E)",
    "Tôi muốn thuyết phục hoặc đàm phán với người khác. (E)",
]
C_cols = [
    "Tôi thích lập kế hoạch, quản lý tài liệu, dữ liệu. (C)",
    "Tôi hứng thú với công việc theo quy trình, quy định rõ ràng. (C)",
]

# Big Five 10 (2 per trait)
O_cols = [
    "Tôi tò mò và thích khám phá những ý tưởng mới. (O)",
    "Tôi thích thử nghiệm nhiều cách tiếp cận khác nhau. (O)",
]
C5_cols = [
    "Tôi luôn hoàn thành công việc đúng hạn. (C)",
    "Tôi thường lên kế hoạch chi tiết cho mọi việc. (C)",
]
E5_cols = [
    "Tôi dễ dàng bắt chuyện với người lạ. (E)",
    "Tôi cảm thấy tràn đầy năng lượng khi ở cùng nhiều người. (E)",
]
A5_cols = [
    "Tôi quan tâm đến cảm xúc của người khác. (A)",
    "Tôi thích hợp tác hơn là cạnh tranh. (A)",
]
N_cols = [
    "Tôi thường hay lo lắng về những việc nhỏ nhặt. (N)",
    "Tôi dễ bị căng thẳng khi gặp áp lực. (N)",
]

# ESSAY + JOBS columns (exact titles)
ESSAY_COL = "Hãy viết một đoạn ngắn (1–2 câu) mô tả sở thích, điểm mạnh, và nghề nghiệp bạn mong muốn trong tương lai."
JOBS_COL = "Nghề nghiệp bạn quan tâm (có thể chọn nhiều)"

# Default language tag for output
DEFAULT_LANGUAGE = "vi"

In [ ]:
import json

import numpy as np
import pandas as pd


def normalize(x):
    try:
        x = float(x)
        if np.isnan(x):
            return np.nan
        return (x - 1.0) / 4.0  # 1..5 -> 0..1
    except:
        return np.nan


def join_jobs(x):
    if pd.isna(x):
        return ""
    s = str(x)
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    return "|".join(parts)


# Load CSV
df = pd.read_csv(INPUT_CSV, encoding="utf-8")


# Add user_id and language
df.insert(0, "user_id", range(1, len(df) + 1))
df["language"] = DEFAULT_LANGUAGE

# Normalize Likert columns
all_groups = [
    R_cols,
    I_cols,
    A_cols,
    S_cols,
    E_cols,
    C_cols,
    O_cols,
    C5_cols,
    E5_cols,
    A5_cols,
    N_cols,
]
for cols in all_groups:
    for c in cols:
        if c not in df.columns:
            raise KeyError(f"Thiếu cột trong CSV: {c}")
        df[c] = df[c].apply(normalize)

# Compute averages
df["R"] = df[R_cols].mean(axis=1)
df["I"] = df[I_cols].mean(axis=1)
df["A"] = df[A_cols].mean(axis=1)
df["S"] = df[S_cols].mean(axis=1)
df["E"] = df[E_cols].mean(axis=1)
df["C"] = df[C_cols].mean(axis=1)

df["O"] = df[O_cols].mean(axis=1)
df["C_big5"] = df[C5_cols].mean(axis=1)
df["E_big5"] = df[E5_cols].mean(axis=1)
df["A_big5"] = df[A5_cols].mean(axis=1)
df["N"] = df[N_cols].mean(axis=1)

# Essay & jobs
df["essay_text"] = df[ESSAY_COL].fillna("").astype(str)
df["target_jobs"] = df[JOBS_COL].apply(join_jobs)

# Instruction fields
df["riasec_instructions"] = RIASEC_INSTRUCTIONS
df["big5_instructions"] = BIG5_INSTRUCTIONS

# Final selection + rename
out = df[
    [
        "user_id",
        "language",
        "essay_text",
        "R",
        "I",
        "A",
        "S",
        "E",
        "C",
        "O",
        "C_big5",
        "E_big5",
        "A_big5",
        "N",
        "target_jobs",
        "riasec_instructions",
        "big5_instructions",
    ]
].rename(columns={"C_big5": "C2", "E_big5": "E2", "A_big5": "A2"})

# Export CSV
out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ Saved -> {OUTPUT_CSV}")

# Export JSONL
with open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
    for _, row in out.iterrows():
        fout.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")
print(f"✅ Saved -> {OUTPUT_JSONL}")

out.head()

In [ ]:
df = pd.read_csv(INPUT_CSV, encoding="utf-8")


# Add user_id and language
df.insert(0, "user_id", range(1, len(df) + 1))
df["language"] = DEFAULT_LANGUAGE

# Normalize Likert columns
all_groups = [
    R_cols,
    I_cols,
    A_cols,
    S_cols,
    E_cols,
    C_cols,
    O_cols,
    C5_cols,
    E5_cols,
    A5_cols,
    N_cols,
]
for cols in all_groups:
    for c in cols:
        if c not in df.columns:
            raise KeyError(f"Thiếu cột trong CSV: {c}")
        df[c] = df[c].apply(normalize)

# Compute averages
df["R"] = df[R_cols].mean(axis=1)
df["I"] = df[I_cols].mean(axis=1)
df["A"] = df[A_cols].mean(axis=1)
df["S"] = df[S_cols].mean(axis=1)
df["E"] = df[E_cols].mean(axis=1)
df["C"] = df[C_cols].mean(axis=1)

df["O"] = df[O_cols].mean(axis=1)
df["C_big5"] = df[C5_cols].mean(axis=1)
df["E_big5"] = df[E5_cols].mean(axis=1)
df["A_big5"] = df[A5_cols].mean(axis=1)
df["N"] = df[N_cols].mean(axis=1)

# Essay & jobs
df["essay_text"] = df[ESSAY_COL].fillna("").astype(str)
df["target_jobs"] = df[JOBS_COL].apply(join_jobs)

# Instruction fields
df["riasec_instructions"] = RIASEC_INSTRUCTIONS
df["big5_instructions"] = BIG5_INSTRUCTIONS

# Final selection + rename
out = df[
    [
        "user_id",
        "language",
        "essay_text",
        "R",
        "I",
        "A",
        "S",
        "E",
        "C",
        "O",
        "C_big5",
        "E_big5",
        "A_big5",
        "N",
        "target_jobs",
        "riasec_instructions",
        "big5_instructions",
    ]
].rename(columns={"C_big5": "C2", "E_big5": "E2", "A_big5": "A2"})

# Export CSV
out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ Saved -> {OUTPUT_CSV}")

# Export JSONL
with open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
    for _, row in out.iterrows():
        fout.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")
print(f"✅ Saved -> {OUTPUT_JSONL}")

out.head()